dataset/nsddata/ppdata/subj01/func1pt8mm/mean.nii.gz brain volume and map/transform it into the same space as sub-01_ses-nsd01_task-nsdcore_run-01_bold.nii.gz

then use that transformation matrix and apply it to the func1pt8mm nsdgeneral mask

this should give you the mask you need

In [8]:
path = "/scratch/gpfs/rk1593/rt_mindEye/masking_operation"
!module load fsl/6.0.6.2 
import os
import nibabel as nib
import numpy as np

flirt is the main program that performs affine registration. The main options are: an input (-in) and a reference (-ref) volume; the calculated affine transformation that registers the input to the reference which is saved as a 4x4 affine matrix (-omat); and output volume (-out) where the transform is applied to the input volume to align it with the reference volume. In addition, FLIRT can also be used to apply a saved transformation to a volume (-applyxfm, -init and -out) or to apply a transform that aligns the NIFTI mm coordinates (-applyxfm, -usesqform and -out; but not -init). For these usages the reference volume must still be specified as this sets the voxel and image dimensions of the resulting volume.

-out outvol -init invol2refvol.mat -applyxfm

# Day 1 Mask

In [3]:
# map mean.nii.gz to sub-01 bold image and make affine
os.system(f"flirt -in /scratch/gpfs/rk1593/rt_mindEye/masking_operation/mean.nii.gz " + 
          "-ref /scratch/gpfs/rk1593/rt_mindEye/masking_operation/sub-01_ses-nsd01_task-nsdcore_run-01_bold.nii.gz " +
          "-omat /scratch/gpfs/rk1593/rt_mindEye/masking_operation/affine_day1_subj1")
# use affine for transforming the mask to the bold image space
os.system(f"flirt -in /scratch/gpfs/rk1593/rt_mindEye/masking_operation/nsdgeneral.nii.gz " + 
          "-ref /scratch/gpfs/rk1593/rt_mindEye/masking_operation/sub-01_ses-nsd01_task-nsdcore_run-01_bold.nii.gz "  +
          "-init /scratch/gpfs/rk1593/rt_mindEye/masking_operation/affine_day1_subj1 " + 
          "-out /scratch/gpfs/rk1593/rt_mindEye/masking_operation/nsdgeneral_transformed_day1_subj1.nii.gz -applyxfm")


0

FLIRT can also be used to apply a saved transformation to a volume (-applyxfm, -init and -out)

In [7]:
# make sure affine of bold image and the transformed mask are the same
ref = nib.load('/scratch/gpfs/rk1593/rt_mindEye/masking_operation/sub-01_ses-nsd01_task-nsdcore_run-01_bold.nii.gz')
mask_nii = nib.load('/scratch/gpfs/rk1593/rt_mindEye/masking_operation/nsdgeneral_transformed_day1_subj1.nii.gz')
print(mask_nii.affine == ref.affine)

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [12]:
#  get mask of various threshold
threshold_list = np.linspace(1e-200,0.999999, 10)
for threshold in threshold_list:
    mask_nii = nib.load('/scratch/gpfs/rk1593/rt_mindEye/masking_operation/nsdgeneral_transformed_day1_subj1.nii.gz')
    mask_np = mask_nii.get_fdata()
    mask_np[np.where(mask_np < threshold)] = 0 
    mask_np[np.where(mask_np >= threshold)] = 1
    print(np.unique(mask_np, return_counts = True))
    new_mask = nib.Nifti1Image(mask_np.astype(float), mask_nii.affine)
    nib.save(new_mask, f'/scratch/gpfs/rk1593/rt_mindEye/masking_operation/thresholded_masks/day1_subj1/mask_transformed_{threshold:.2f}_day1_subj1.nii.gz')

(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1194631,   14969]))
(array([0., 1.]), array([1195201,   14399]))
(array([0., 1.]), array([1195803,   13797]))
(array([0., 1.]), array([1196444,   13156]))
(array([0., 1.]), array([1197256,   12344]))
(array([0., 1.]), array([1198195,   11405]))
(array([0., 1.]), array([1199366,   10234]))
(array([0., 1.]), array([1201009,    8591]))
(array([0., 1.]), array([1204730,    4870]))


flirt -in newvol -ref refvol -out outvol -init invol2refvol.mat -applyxfm

[[-1.79999995e+00  1.79999994e-16 -1.91275342e-17  1.07152489e+02]
 [ 1.79999994e-16  1.78980815e+00 -1.91275328e-01 -6.96683807e+01]
 [ 0.00000000e+00  1.91275313e-01  1.78980827e+00 -5.89975739e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [21]:
import numpy as np
threshold_list =  np.linspace(0,1, 100)
mask_np[np.where(mask_np < 0.9)] = 0 
mask_np[np.where(mask_np >= 0.9)] = 1

In [24]:
np.unique(mask)
new_mask = nib.Nifti1Image(mask_np.astype(float), mask_nii.affine)

In [25]:
nib.save(new_mask, '/scratch/gpfs/rk1593/rt_mindEye/masking_operation/mask_transformed_0.9.nii.gz')

In [29]:
np.unique(mask_np, return_counts = True)[1][1]

8401

In [54]:

threshold_list = np.linspace(1e-200,1e-199, 100)
for threshold in threshold_list:
    mask_nii = nib.load('/scratch/gpfs/rk1593/rt_mindEye/masking_operation/nsdgeneral_transformed.nii.gz')
    mask_np = mask_nii.get_fdata()
    mask_np[np.where(mask_np < threshold)] = 0 
    mask_np[np.where(mask_np >= threshold)] = 1
    print(np.unique(mask_np, return_counts = True))
    if np.unique(mask_np, return_counts = True)[1][1] == 15724:
        print("yes!", threshold)

(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0., 1.]), array([1193926,   15674]))
(array([0.

KeyboardInterrupt: 